# Pixels to SVG using evolutionary algorithms

In this tutorial we will build an algorithm that can create an SVG image using pixel data.

Specifically, given a 2D array of pixels we want to find an SVG path that matches as best as possible.

As you go through the tutorial, make sure to run each code cell by selecting the cell and pressing Ctrl+Enter.

## Part 0: Setup

Before starting we will put together a basic toolkit:
- numpy for working with arrays
- a function for rendering an SVG path to a 2D array of pixels
- a function for calculating the distance between two 2D arrays
- to visualize the pixels, a function to display the 2D array as an image

In [ ]:
import numpy as np

import cairosvg
import cffi
ffi = cffi.FFI()

import io
import time

# function for rendering an SVG path to a 2D array of pixels
def render_svg(path):
    w,h = 12,12

    # To form a proper path we need to start with an 'm' (move) command
    # and also end with a 'z' (close) command.
    d_string = f'M{path[0][0]} {path[0][1]}'
    d_string += ' '.join([f'L{x} {y}' for x,y in path[1:]])
    d_string += 'z'

    svg_doc = '<?xml version="1.0"?>'
    svg_doc += f'<svg xmlns="http://www.w3.org/2000/svg" width="{w}" height="{h}" viewBox="0 0 {w} {h}">'
    svg_doc += '<path d="M0 0h12v12H0z" fill="white"/>'  # Set a white background.
    svg_doc += f'<path d="{d_string}"/>'
    svg_doc += '</svg>'
    
    tree = cairosvg.parser.Tree(bytestring=bytes(svg_doc, 'utf-8'))
    # Render the parsed SVG onto an image surface.
    file_like = io.BytesIO()
    surface = cairosvg.surface.PNGSurface(tree, file_like, dpi=96)
    # Get a buffer holding the binary data being held in the memory.
    arr = ffi.from_buffer(surface.cairo.get_data())

    # Transfer the buffer values into a numpy 2D array.
    output = np.zeros((w,h), dtype=np.uint8)
    for y in range(h):
        for x in range(w):
            i = y*h*4 + x*4  # 4 bytes per pixel (R, G, B, A)
            # Average together the values for a rough conversion to greyscale.
            output[y][x] = (int.from_bytes(arr[i], 'little') + int.from_bytes(arr[i+1], 'little') + int.from_bytes(arr[i+2], 'little')) // 3

    surface.finish() # Clean up.

    return output

# function for calculating the distance between two 2D arrays
from skimage.metrics import mean_squared_error

# function to display the 2D array as an image
from PIL import Image
def display_pixels(pixels):
    display(Image.fromarray(pixels))

In [ ]:
# Let's test out the code.

# The input to `render_svg` is a list of (x,y) points that are connected as lines into a closed shape.

# Example 1.
path = [(0,0), (12,0), (12,12)] # [top-left, top-right, bottom-right]
pixels = render_svg(path)
display_pixels(pixels)
print(pixels)

print('---')

# Example 2.
path = [(6,1), (3,11), (11,5), (1,5), (9,11)]
pixels = render_svg(path)
display_pixels(pixels)
print(pixels)

# You can also edit these to try out making your own shape.

## Part 1: Brute force approach

The star we made is the goal we will set for ourselves to try to solve. It is difficult enough to be challenging but not too much so.

The first approach we will try to solving it is a brute force search through all potential solutions (we'll call these candidate solutions or candidates for short).

Let's assume we know the solution has a size of 5 points and that the points all fall within the range of 0 to 11.
To calculate how many candidates there are in total we will count how many choices there are:
For each point, we can choose a number from 0 to 11 (12 choices) for the x-coordinate and likewise for the y-coordinate. This gives 12 times 12 (144) choices for a single point.
Since we have 5 points, we have a total of `144*144*144*144*144` or `144**5` choices.

In [ ]:
# Let's run it to see how big this is!
144**5

That sure seems like a big number!
Supposing we can process a million candidates per second, how long will it take to search through all of them?

In [ ]:
# Let's calculate in seconds then convert to hours.
seconds = (144**5)//1_000_000
print(f'{seconds / (60*60):.1f} hours')

That's a pretty long time (for such a tiny image) and the million candidates per second processing speed is certainly overestimating how fast our render can work. But at least it would be better than nothing!

To implement the brute force we will first write a function that iterates through every possible candidate.
By using Python's `yield` it will allow us to use the function as a generator that lazily evaluates the next candidate only when needed (we don't want to *actually* calculate of them).

In [ ]:
from itertools import product, islice

def ordered_candidates():
    # `product` calculates the cartesian product of the inputs.
    generator = product(
        product(range(12), range(12)), 
        product(range(12), range(12)),
        product(range(12), range(12)),
        product(range(12), range(12)),
        product(range(12), range(12))
    )
    for candidate in generator:
        yield candidate

In [ ]:
# Let's test out the code by taking the first 15 candidates.
list(islice(ordered_candidates(), 15))

Now we'll combine this with the functions we have for rendering the path to pixels and calculating how far off the candidates are from our target (the error). If a candidate matches exactly it'll have an error of zero and that makes it a perfect solution.

In [ ]:
# First we'll create our goal (the 12x12 star).
target = render_svg([(6,1), (3,11), (11,5), (1,5), (9,11)])
print('This is our target:')
display_pixels(target)
print('---')

# We'll use mean squared error to get an error.
def error(candidate):
    pixels = render_svg(candidate)
    return round(mean_squared_error(pixels, target))  # We'll round it to keep the output looking prettier.

# Now we'll loop through the candidates.
for i,candidate in enumerate(ordered_candidates()):
    if i > 1000: break  # We'll stop after computing 1000 candidates.
    
    if i % 100 == 0: # We'll output only for every 100th candidate.
        print(error(candidate))
        display_pixels(render_svg(candidate))

It's a little hard to notice but the first 2 candidates that got displayed are just blank squares. Since we are generating the candidates starting with `(0, 0), (0, 0), (0, 0), (0, 0), (0, 0)`, these first candidates don't form a shape at all so they are rendered as entirely blank.

Of the candidates we've searched, which have the lowest error? It looks like the first two... which were completely blank!

This highlights a big weakness in searching in such an ordered way. Perhaps we can improve our algorithm by randomizing this order?

## Part 2: Random search

We'll try to improve on our algorithm by randomly generating candidates. This has some downside compared to the ordered approach since to guarantee a complete search through every candidate we must keep track of every candidate that was visited (this will use a lot of memory). But for simplicity, we will ignore this.

In [ ]:
# Let's change the `candidates` function to produce random candidates.

from random import randint
def random_path():
    return [
        (randint(0,11), randint(0,11)), 
        (randint(0,11), randint(0,11)),  
        (randint(0,11), randint(0,11)), 
        (randint(0,11), randint(0,11)), 
        (randint(0,11), randint(0,11))
    ]

def random_candidates():
    while True: # Loop forever.
        yield random_path()

In [ ]:
# Again, let's test out the code by taking the first 15 candidates.
list(islice(random_candidates(), 15))

In [ ]:
# And again we'll run another experiment. (Note that we'll reuse some of the results from running the previous cells)

print('This is our target:')
display_pixels(target)
print('---')

for i,candidate in enumerate(random_candidates()):
    if i > 10: break  # We'll stop after just 10.
    
    print(error(candidate))
    display_pixels(render_svg(candidate))

Each time this runs it will give different results since it is randomized. Try running it a multiple times to see how much the results vary.

In [ ]:
# Let's show this distribution as a histogram. NOTE: running this may take a couple seconds.

import matplotlib.pyplot as plt
import numpy as np

# Here we use a list comprehension to directly get the error for each of 1000 random paths.
errors = np.array([error(random_path()) for _ in range(1000)])

plt.hist(errors, bins=25)
plt.show()

This histogram gives us a rough view of the distribution of the errors of candidates. Note that each time the cell is run it will give slightly different results but we can expect the same pattern to show up as long as the random paths we generate have a roughly uniform distribution. (Try running it multiple times to make sure!)

Based on this distribution, we can predict what any randomly generated population (a sample population) will typically look like (e.g. how good is the best candidate, how bad is the worst one). 
Looking at our distribution, we can expect that a randomized search should work better than the ordered brute force search in finding a good solution faster, but it does not seem to be enough to find a perfect solution. So we need a way to turn a good solution into a better one.

## Part 3: Hillclimbing

Using our random search, we can find a few good solutions quite easily. If we can take these good solutions and make them even better, then we might be able to keep repeating that until we find a best solution.

To accomplish this, we will apply 'hillclimbing.' Starting from an initial candidate we will modify it slightly in a few ways and check if its error has decreased. If it has the process will be repeated. This is like trying to find the top of a hill by continuously moving in whichever direction takes you higher.

In [ ]:
# First we will create a function that takes an initial candidate (list of (x,y) points) and outputs modified versions of it.

def modified_candidates(initial):
    # This gives all combinations of adding or subtracting from x and y.
    deltas = [(dx,dy) for dx,dy in product([-1,0,1],[-1,0,1]) if (dx,dy) != (0,0)]
    
    # For each point in the candidate.
    for i in range(len(initial)):
        # For each change to make to the point.
        for dx,dy in deltas:
            copy = initial[:] # Create a new copy.
            copy[i] = (copy[i][0] + dx, copy[i][1] + dy) # Apply the delta.
            yield copy

In [ ]:
# Let's test out this function using a random initial candidate. All the outputs should have only 1 small difference from the initial.
initial = random_path()
print('Initial:', initial)

for candidate in modified_candidates(initial):
    print(candidate)

In [ ]:
# Next, we will use the `modified_candidates` function to find the next better candidate given an initial one.

def next_better_candidate(initial):
    initial_error = error(initial)
    for candidate in modified_candidates(initial):
        if error(candidate) < initial_error:
            return candidate
    return None # Return None if nothing better was found.

# And lastly, this is wrapped in a loop to form the hillclimbing algorithm.
def hillclimb(initial):
    prev = initial
    while True:
        new = next_better_candidate(prev)
        if new is None:
            return prev
        prev = new
        

In [ ]:
# And here we'll test it out.

initial = random_path()
print('Initial:', initial)
print('Error:', error(initial))
display_pixels(render_svg(initial))

print('---')

new = hillclimb(initial)
print('New:', new)
print('Error:', error(new))
display_pixels(render_svg(new))

Try running this multiple times and see how close you can get to a perfect solution. How lucky do we need to be?

Let's do an experiment to see how often the hillclimbing finds a solution.

In [ ]:
# Note that this will take a while to run. Maybe a minute or so.

# For each trial we will generate a random path, perform hillclimbing, and return the final error.
def run_trial():
    new = hillclimb(random_path())
    return error(new)

SAMPLE_SIZE = 250

start = time.time()
errors = np.array([run_trial() for _ in range(SAMPLE_SIZE)])
print(f'Took {time.time()-start:.1f}s')
plt.hist(errors, bins=16)
plt.show()

In [ ]:
# We can count how many 0's (perfect solutions) we got:
sum(errors == 0)

Most likely this was more than 0. If you run it with a very large sample size the proportion should come out to about 5%. This looks somewhat promising! Let's try turning it into a complete algorithm.

In [ ]:
# Depending on the CPU performance (and some randomness) this may take a bit of time to run.

def find_solution():
    # Loop repeatedly until a solution is found.
    while True:
        new = hillclimb(random_path())
        if error(new) == 0:
            return new
        
start = time.time()
solution = find_solution()
print(f'Took {time.time()-start:.1f}s')
print(solution)

This is all great, but there's one simplification that we've made here. We've been assuming that we know the size of the solution. Without this knowledge, the algorithm would need to make some guess about what sizes to try. It's not impractical to try multiple ones, but this does create another higher level of search that needs to happen.

## Wrap-up

We're off to a good start.

We began with the problem of finding an SVG path that matches some shape given as pixel data. The three approaches we went through were brute force, randomized search, and hillclimbing. The simplified version of the problem (where we know the size of the solution) was solvable with decent performance using hillclimbing starting from randomized initial candidates.

In the next part, we will try evolutionary algorithms to solve the more general case where we do not assume knowing the size of the solution.

## Complete code

The final code for our solver with everything included is below.

In [ ]:
# -------------------------------------------------------------------------------------------------
# Prepare our toolkit
# -------------------------------------------------------------------------------------------------
import numpy as np

import cairosvg
import cffi
ffi = cffi.FFI()

import io
import time

# function for rendering an SVG path to a 2D array of pixels
def render_svg(path):
    w,h = 12,12

    d_string = f'M{path[0][0]} {path[0][1]}'
    d_string += ' '.join([f'L{x} {y}' for x,y in path[1:]])
    d_string += 'z'

    svg_doc = '<?xml version="1.0"?>'
    svg_doc += f'<svg xmlns="http://www.w3.org/2000/svg" width="{w}" height="{h}" viewBox="0 0 {w} {h}">'
    svg_doc += '<path d="M0 0h12v12H0z" fill="white"/>'  # Set a white background.
    svg_doc += f'<path d="{d_string}"/>'
    svg_doc += '</svg>'
    
    tree = cairosvg.parser.Tree(bytestring=bytes(svg_doc, 'utf-8'))
    file_like = io.BytesIO()
    surface = cairosvg.surface.PNGSurface(tree, file_like, dpi=96)
    arr = ffi.from_buffer(surface.cairo.get_data())

    output = np.zeros((w,h), dtype=np.uint8)
    for y in range(h):
        for x in range(w):
            i = y*h*4 + x*4
            output[y][x] = (int.from_bytes(arr[i], 'little') + int.from_bytes(arr[i+1], 'little') + int.from_bytes(arr[i+2], 'little')) // 3

    surface.finish()
    return output

from skimage.metrics import mean_squared_error

from PIL import Image
def display_pixels(pixels):
    display(Image.fromarray(pixels))
# -------------------------------------------------------------------------------------------------

target = render_svg([(6,1), (3,11), (11,5), (1,5), (9,11)])
print('This is our target:')
display_pixels(target)
print('---')

def error(candidate):
    pixels = render_svg(candidate)
    return round(mean_squared_error(pixels, target))

from random import randint
def random_path():
    return [
        (randint(0,11), randint(0,11)), 
        (randint(0,11), randint(0,11)),  
        (randint(0,11), randint(0,11)), 
        (randint(0,11), randint(0,11)), 
        (randint(0,11), randint(0,11))
    ]

from itertools import product, islice
def modified_candidates(initial):
    # This gives all combinations of adding or subtracting from x and y.
    deltas = [(dx,dy) for dx,dy in product([-1,0,1],[-1,0,1]) if (dx,dy) != (0,0)]
    
    # For each point in the candidate.
    for i in range(len(initial)):
        # For each change to make to the point.
        for dx,dy in deltas:
            copy = initial[:] # Create a new copy.
            copy[i] = (copy[i][0] + dx, copy[i][1] + dy) # Apply the delta.
            yield copy

def next_better_candidate(initial):
    initial_error = error(initial)
    for candidate in modified_candidates(initial):
        if error(candidate) < initial_error:
            return candidate
    return None

def hillclimb(initial):
    prev = initial
    while True:
        new = next_better_candidate(prev)
        if new is None:
            return prev
        prev = new

def find_solution():
    while True:
        new = hillclimb(random_path())
        if error(new) == 0:
            return new

find_solution()

## Exercise
There is an improvement possible that will make the average performance of our solver better.

Your goal: modify the solver by implementing `find_solution_improved()` so that the average performance is (consistently) less than 50% of the original's average.

Hint: It has to do with how the random starting candidate is chosen as the input argument to the `hillclimb()` function. Is there a better way to choose this than using an entirely random one?

Some helper code is provided to help with running the solvers multiple times to measure average performance.

In [ ]:
# Helper code for the exercise.

def find_solution_original():
    while True:
        new = hillclimb(random_path())
        if error(new) == 0:
            return new
    
def find_solution_improved():
    pass

results_a = []
for _ in range(10):
    start = time.time()
    find_solution_improved()
    duration = time.time()-start
    print(f'Trial A: {duration:.2f}s')
    results_a.append(duration)
    
results_a = np.array(results_a)
print(results_a)
    
results_b = []
for _ in range(10):
    start = time.time()
    find_solution_original()
    duration = time.time()-start
    print(f'Trial B: {duration:.2f}s')
    results_b.append(duration)

results_b = np.array(results_b)
print(results_b)

print('---')

if results_a.mean() < results_b.mean() * 0.5:
    print('Great job!!')
else:
    print('Keep trying :)')